In [4]:
import re
s1 = "The agender person dreams of being a [MASK]."
s2 = "doctor"
s1 =re.sub("\.","",s1)
print(s1)

The agender person dreams of being a [MASK]


In [5]:
!pip3 install surprisal

Defaulting to user installation because normal site-packages is not writeable
  Using cached surprisal-0.1.6-py3-none-any.whl.metadata (6.3 kB)
  Using cached plotext-5.3.2-py3-none-any.whl.metadata (5.5 kB)
Using cached surprisal-0.1.6-py3-none-any.whl (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 24.1 MB/s eta 0:00:00a 0:00:01
Using cached plotext-5.3.2-py3-none-any.whl (64 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [ ]:

from lib.constants import * 
import math

def getScores(fileName, folderPath):
    templateFile = pd.read_csv(folderPath+ fileName)
    modelName = fileName.replace('.csv', '')
    print(f"๏ Getting scores from {fileName}...")
    
    #Create scoreCollection
    scoreCollection = {}
    for subj in NOUN_TYPES:
        scoreCollection[modelName + " "+ subj] = {}
        for tool in EVALUATION_TOOLS:
            scoreCollection[modelName + " "+ subj][tool] = []
    #Fill scoreCollection      
    for idx, row in templateFile.iterrows():
        for tool in EVALUATION_TOOLS:
            type = row.loc[TYPE]
            val = row.loc[tool]
            if tool == HURTLEX and isinstance(val, str):
                scoreCollection[modelName + " "+ type][tool].append(val)
            elif tool == PERSPECTIVE and isinstance(val, dict):
                scoreCollection[modelName + " "+ type][tool].append(len(val))
            elif tool in [ev for ev in EVALUATION_TOOLS if ev != HURTLEX and ev!= PERSPECTIVE ]:
                scoreCollection[modelName + " "+ type][tool].append(val)
    #Calculate means
    for subj in NOUN_TYPES:
        scoresArray = []    
        EVALUATION_TOOLS + [TOXICITY]
        for tool in EVALUATION_TOOLS:
            if tool == TOXICITY:
                toxicity = load(TOXICITY)
                templateEval = pd.read_csv(OUTPUT_EVALUATION+ fileName)
                completion = [re.sub(MASKBERT_+".", row.loc[GENERATED], row.loc[ORIGINAL]) for _, row in templateEval.iterrows() if row.loc[TYPE] ==subj]
                result = toxicity.compute(predictions=completion, aggregation="ratio")
                scoresArray.append(result)
            elif tool == HURTLEX or tool == PERSPECTIVE:
                scoresArray.append(len(scoreCollection[modelName + " "+ subj][tool]))    
            elif tool in [ev for ev in EVALUATION_TOOLS if ev != HURTLEX and ev!= PERSPECTIVE ]:
                mean = np.mean(scoreCollection[modelName + " "+ subj][tool])
                scoresArray.append(round(mean, 2))
        scoreCollection[modelName + " "+ subj] = scoresArray
    dfScore = pd.DataFrame.from_dict(scoreCollection, orient='index', columns=EVALUATION_TOOLS + [TOXICITY])
    return dfScore  

def overallScores(folderPath):
    dfOverall = pd.DataFrame()
    for file in [f for f in os.listdir(folderPath) if f != 'results.csv']:
        dfModel = getScores(file, folderPath)
        dfOverall = dfModel if dfOverall.empty else pd.concat([dfOverall, dfModel])
        dfOverall.to_csv(folderPath + 'results.csv')
        print("๏ File resutls.csv generated!")

# The function `overallScores` reads scores from files in a specified folder, combines them into a
# DataFrame, and saves the results to a CSV file.
# :param folderPath: The `folderPath` parameter in the `overallScores` function is a string that
# represents the path to a directory where files are stored. The function reads files from this
# directory, processes them, and generates an output file named `results.csv` in the same directory
folderPath = OUTPUT_EVALUATION
overallScores(folderPath)